In [1]:
!git clone https://github.com/dsloet/StyleTransfer.git

Cloning into 'StyleTransfer'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 38 (delta 10), reused 19 (delta 3), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [2]:
%cd StyleTransfer/

/content/StyleTransfer


In [0]:
%mkdir style_tensorflow/output_images

In [4]:
import urllib.request
urllib.request.urlretrieve('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5', 'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
 <http.client.HTTPMessage at 0x7fe1a091d320>)

In [5]:
%tensorflow_version 1x

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1x`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [0]:
from style_tensorflow.StyleTransferTF import get_content_loss, gram_matrix, get_style_loss, get_feature_representations
from style_tensorflow.StyleTransferTF import compute_loss

In [0]:
from style_tensorflow.utils_tf import load_img, deprocess_img

In [8]:
import tensorflow as tf
from tensorflow.python.keras.preprocessing import image as kp_image

# Keras is only used to load VGG19 model as a high level API to TensorFlow 
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras import backend as K

# pillow is used for loading and saving images
from PIL import Image

# numPy is used for manipulation of array of object i.e Image in our case
import numpy as np

Using TensorFlow backend.


In [0]:
# list of layers to be considered for calculation of Content and Style Loss
content_layers = ['block3_conv3']
style_layers   = ['block1_conv1','block2_conv2','block4_conv3']

num_content_layers = len(content_layers)
num_style_layers   = len(style_layers)

# path where the content and style images are located
content_path = 'images/dancing.jpg'
style_path   = 'images/picasso.jpg'

# Save the result as
save_name = 'dance_picasso.jpg'

# path to where Vgg19 model weight is located 
vgg_weights = "vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5" 

In [0]:
# Using Keras Load VGG19 model
def get_model(content_layers,style_layers):

  # Load our model. We load pretrained VGG, trained on imagenet data
  vgg19           = VGG19(weights=None, include_top=False)

  # We don't need to (or want to) train any layers of our pre-trained vgg model, so we set it's trainable to false.
  vgg19.trainable = False

  style_model_outputs   =  [vgg19.get_layer(name).output for name in style_layers]
  content_model_outputs =  [vgg19.get_layer(name).output for name in content_layers]
  
  model_outputs = content_model_outputs + style_model_outputs

  # Build model 
  return Model(inputs = vgg19.input, outputs = model_outputs),  vgg19

In [0]:
def run_style_transfer(content_path, style_path, num_iterations=200, content_weight=0.1, style_weight=0.9): 

  # Create a tensorflow session 
  sess = tf.Session()

  # Assign keras back-end to the TF session which we created
  K.set_session(sess)

  model, vgg19 = get_model(content_layers,style_layers)

  # Get the style and content feature representations (from our specified intermediate layers) 
  style_features, content_features = get_feature_representations(model, content_path, style_path, num_content_layers)
  gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]

  # VGG default normalization
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means 
    

  # In original paper, the initial stylized image is random matrix of same size as that of content image
  # but in later images content image was used instead on random values for first stylized image
  # because it proved to help to stylize faster
  generated_image = load_img(content_path)
  # generated_image = np.random.randint(0,255, size=generated_image.shape) 
  
  # Create tensorflow variable to hold a stylized/generated image during the training 
  generated_image = tf.Variable(generated_image, dtype=tf.float32)

  model_outputs = model(generated_image)

  # weightages of each content and style images i.e alpha & beta
  loss_weights = (style_weight, content_weight)

  # Create our optimizer
  loss = compute_loss(model, loss_weights, model_outputs, gram_style_features, content_features, num_content_layers, num_style_layers)
  opt = tf.train.AdamOptimizer(learning_rate=9, beta1=0.9, epsilon=1e-1).minimize( loss[0], var_list = [generated_image])

  sess.run(tf.global_variables_initializer())
  sess.run(generated_image.initializer)
  
  # loading the weights again because tf.global_variables_initializer() resets the weights
  vgg19.load_weights(vgg_weights)


  # Put loss as infinity before training starts and Create a variable to hold best image (i.e image with minimum loss)
  best_loss, best_img = float('inf'), None

  for i in range(num_iterations):

    # Do optimization
    sess.run(opt)

    # Make sure image values stays in the range of max-min value of VGG norm 
    clipped = tf.clip_by_value(generated_image, min_vals, max_vals)
    # assign the clipped value to the tensor stylized image
    generated_image.assign(clipped)


    # Open the Tuple of tensors 
    total_loss, style_score, content_score = loss
    total_loss = total_loss.eval(session=sess)


    if total_loss < best_loss:

      # Update best loss and best image from total loss. 
      best_loss = total_loss

      # generated image is of shape (1, h, w, 3) convert it to (h, w, 3)
      temp_generated_image = sess.run(generated_image)[0]
      best_img = deprocess_img(temp_generated_image)

      s_loss = sess.run(style_score)
      c_loss = sess.run(content_score)

      # print best loss
      print('best: iteration: ', i ,'loss: ', total_loss ,'  style_loss: ',  s_loss,'  content_loss: ', c_loss)

    # Save image after every 2 iterations 
    if (i+1)%10 == 0:
      output = Image.fromarray(best_img)
      output.save('style_tensorflow/output_images/'+str(i+1)+'-'+save_name)

  # after num_iterations iterations are completed, close the TF session 
  sess.close()
      
  return best_img, best_loss



In [12]:
best, best_loss = run_style_transfer(content_path, style_path, num_iterations=200, content_weight=0.1)








best: iteration:  0 loss:  35113504.0   style_loss:  39013160.0   content_loss:  16622.215
best: iteration:  1 loss:  31993560.0   style_loss:  35546330.0   content_loss:  18658.658
best: iteration:  2 loss:  21476932.0   style_loss:  23861366.0   content_loss:  17031.695
best: iteration:  3 loss:  16614883.0   style_loss:  18459168.0   content_loss:  16323.768
best: iteration:  4 loss:  13318336.0   style_loss:  14796210.0   content_loss:  17456.623
best: iteration:  5 loss:  10285772.0   style_loss:  11426581.0   content_loss:  18504.496
best: iteration:  6 loss:  8041367.0   style_loss:  8932766.0   content_loss:  18781.05
best: iteration:  7 loss:  6631909.5   style_loss:  7366650.5   content_loss:  19243.39
best: iteration:  8 loss:  4929805.5   style_loss:  5475359.0   content_loss:  19823.004
best: iteration:  9 loss:  3906299.0   style_loss:  4338063.0   content_loss:  20425.3
best: iteration:  10 loss:  3079191.8   style_loss:  3419010.0   content_loss:  20829.076
best:

KeyboardInterrupt: ignored